In [ ]:
import os
import sys
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from interpret.glassbox import ExplainableBoostingClassifier
import gamchanger as gc

import random

from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot as plt
from json import load
import joblib

sys.path.append(os.path.expanduser("~/CO2-to-C3/src"))
from ml_tools import filter_max_jtarget
from gamchanger_tools import create_gam_feature_df
from utils.help_utils import get_data_df

import warnings
warnings.filterwarnings('ignore')

data_path = "~/data"   # Directory path where you stored the data – Dataset available at https://doi.org/10.5281/zenodo.15107045

pb_df = get_data_df(data_path, "Pourbaix_phase.xlsx")
ocp_df = get_data_df(data_path, "OCP_Eads.xlsx")
all_compositions_featurized_df = get_data_df(data_path, "Compositions_featurized.csv", ['e1', 'e2', 'e3'])
co2r_df = get_data_df(data_path, "CO2R_dataset.xlsx")

## [STEP #0] Preprocess CO2R data

In [ ]:
co2r_df[f'j_C3H6'] = co2r_df['current_density'] * co2r_df['FE_C3H6_mean'] / 100
co2r_df_processed = filter_max_jtarget(co2r_df, 'C3H6')

## [STEP #1] EBM model training

In [ ]:
# Set TRAINing Parameters
batch_train = [0,]      # e.g., [0] or [0, 1] .... [0, 1, 2, 3, 4, 5].  Batch 0 means the seed dataset
cutoff = 0.05           # Target product (C3H6) cut-off crrent density (see Supplementary Fig. 3a bottom)

millers = [(1,0,0), (2,1,1)]
species_columns = ['*OCHO','*COOH','CO*COH-2*CO','*CHO-*CO','*C-*CHO']

random_seed = 10

train_cols = ['e1', 'e2', 'e3'] + ['f1', 'f2', 'f3'] + ['has_e1', 'has_e2', 'has_e3'] +\
             ['ele1_' + x for x in species_columns] + ['ele2_' + x for x in species_columns] + ['ele3_' + x for x in species_columns]

In [ ]:
# Train DataFrame

co2r_df_train = co2r_df_processed[co2r_df_processed['batch'].isin(batch_train)]
print(f"Train size: {co2r_df_train['composition'].nunique()}")

feature_df = create_gam_feature_df(co2r_df_train, ocp_df, millers = millers)

train_df = pd.merge(feature_df, co2r_df_train, on='composition')

train_df['j_C3H6_log10'] = train_df['j_C3H6'].apply(lambda x: np.nan if x == 0 else np.log10(x))
train_df['j_binary'] = train_df['j_C3H6_log10'].apply(lambda x: True if x > np.log10(cutoff) else False)
# Properly fillna for GAM
selected_columns = [col for col in train_df.columns if col not in ['e1', 'e2', 'e3', 'j_C3H6']]
train_df[selected_columns] = train_df[selected_columns].fillna(0)

X_train = train_df[train_cols].copy()
y_train = train_df['j_binary'].copy()

train_df.head()

In [ ]:
# Define hyperparameter search space
param_grid = {
    'max_leaves': [2, 3],
    'smoothing_rounds': [50, 75],
    'learning_rate': [0.005, 0.015],
    'interactions': [0, 0.9]
}

# Define the EBM model
ebm = ExplainableBoostingClassifier(feature_names=train_cols, n_jobs=-2, random_state=random_seed)

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=ebm,
    param_grid=param_grid,
    scoring='f1',  # Use accuracy as the scoring metric
    cv=5,  # 5-fold cross-validation
    n_jobs=-1,  # Use all available CPUs
    verbose=2  # Display progress during the search
)

# Fit GridSearchCV on the training data
grid_search.fit(X_train, y_train)

# Retrieve the best parameters and score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Train the best model on the entire training dataset
best_ebm = grid_search.best_estimator_

# Save the best model
joblib.dump(best_ebm, f"./EBM_models/ebm_trainBATCH{batch_train}.joblib")

# Output the best parameters and scores
print("Best Parameters:", best_params)
print("Best Cross-Validation Score:", best_score)

## [STEP #2] Visualize & Edit & Save - EBM model

In [ ]:
# GAM Visualization & Edit Feature Importance & Save Edited Model
best_ebm = joblib.load(f"./ebm_models/ebm_trainBATCH{batch_train}.joblib")
ebm_weights_dict = gc.get_model_data(best_ebm)
gc.visualize(best_ebm, X_train.to_numpy(), y_train.to_numpy())

'''
Detailed shape function edit records are in Supplementary Note 5 and Supplementary Table 3.

Save the edited model with name of 'trainBATCH{batch_train}_EDITED.gamchanger'
e.g., 'trainBATCH[0]_EDITED.gamchanger' or 'trainBATCH[0, 1, 2, 3, 4, 5]_EDITED.gamchanger'
'''

## [STEP #3] Get Next Batch Suggestion

In [ ]:
# Load the edited EBM model
ebm_weights_dict_edited = load(open(f"./gamchanger_files/trainBATCH{batch_train}_EDITED.gamchanger"))
best_ebm_edited = gc.get_edited_model(best_ebm, ebm_weights_dict_edited)

In [ ]:
# Get predictions for all compositions
prediction_df = all_compositions_featurized_df.copy()
# Properly fillna for GAM
selected_columns = [col for col in prediction_df.columns if col not in ['e1', 'e2', 'e3']]
prediction_df[selected_columns] = prediction_df[selected_columns].fillna(0)

prediction_df['j_binary_pred'] = best_ebm_edited.predict(prediction_df[train_cols])
prediction_df.head()

In [ ]:
nextBatch_suggestion_df = prediction_df[['composition_nominal', 'j_binary_pred']]

nextBatch_suggestion_df.to_csv(f"./nextBatch_suggestion/BATCH[{max(batch_train)+1}]_suggestion.csv", index=False)
print(f'Batch{max(batch_train)+1} suggestion saved.')

## [STEP #4] Run HT-Experiment (based on the selected compositions)

## [STEP #5] EBM Model Comparison (based on the new Batch Data)

In [ ]:
# Set TESTing Parameters
batch_new = 1

In [ ]:
# TEST DataFrame

co2r_df_test = co2r_df_processed[co2r_df_processed['batch'] == batch_new]
print(f"Test size: {co2r_df_test['composition'].nunique()}")

feature_df = create_gam_feature_df(co2r_df_test, ocp_df, millers = millers)

test_df = pd.merge(feature_df, co2r_df_test, on='composition')

test_df['j_C3H6_log10'] = test_df['j_C3H6'].apply(lambda x: np.nan if x == 0 else np.log10(x))
test_df['j_binary'] = test_df['j_C3H6_log10'].apply(lambda x: True if x > np.log10(cutoff) else False)
# Properly fillna for GAM
selected_columns = [col for col in test_df.columns if col not in ['e1', 'e2', 'e3', 'j_C3H6']]
test_df[selected_columns] = test_df[selected_columns].fillna(0)

X_test = test_df[train_cols].copy()
y_test = test_df['j_binary'].copy()

test_df.head()

### 1. Predicted vs. Actual Value: w/o Domain Knowledge-based edit

In [ ]:
### Tested on Test Batch
y_pred_test_wo_edit = best_ebm.predict(X_test)

cm = confusion_matrix(y_test, y_pred_test_wo_edit)
roc_auc = roc_auc_score(y_test, y_pred_test_wo_edit)
f1 = f1_score(y_test, y_pred_test_wo_edit)
accuracy = accuracy_score(y_test, y_pred_test_wo_edit)

fig, (ax_metrics, ax_cm) = plt.subplots(2, 1, figsize=(8, 8), gridspec_kw={'height_ratios': [1, 3]})

ax_metrics.axis('off')
ax_metrics.text(0.5, 0.5, f'ROC AUC: {roc_auc:.3f}\nF1 Score: {f1:.3f}\nAccuracy: {accuracy:.3f}', 
                horizontalalignment='center', verticalalignment='center', fontsize=20)

# Plot confusion matrix
im = ax_cm.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
ax_cm.set_title(f'Confusion Matrix', fontsize=18)

# Add value annotations
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax_cm.text(j, i, format(cm[i, j], 'd'),
                   ha="center", va="center",
                   color="white" if cm[i, j] > thresh else "black")

# Set labels
ax_cm.set_xlabel('Predicted label', fontsize=12)
ax_cm.set_ylabel('Actual label', fontsize=12)
ax_cm.set_xticks([0, 1])
ax_cm.set_yticks([0, 1])
ax_cm.set_xticklabels(['False', 'True'], fontsize=12)
ax_cm.set_yticklabels(['False', 'True'], fontsize=12)

# Add colorbar
plt.colorbar(im, ax=ax_cm)
plt.tight_layout()
plt.show()

### 2. Predicted vs. Actual Value: after Domain Knowledge-based edit

In [ ]:
# Prediction
y_pred_test_edited = best_ebm_edited.predict(X_test)

# Analysis
cm = confusion_matrix(y_test, y_pred_test_edited)
roc_auc = roc_auc_score(y_test, y_pred_test_edited)
f1 = f1_score(y_test, y_pred_test_edited)
accuracy = accuracy_score(y_test, y_pred_test_edited)

# Plot
fig, (ax_metrics, ax_cm) = plt.subplots(2, 1, figsize=(8, 8), gridspec_kw={'height_ratios': [1, 3]})

ax_metrics.axis('off')
ax_metrics.text(0.5, 0.5, f'ROC AUC: {roc_auc:.3f}\nF1 Score: {f1:.3f}\nAccuracy: {accuracy:.3f}', 
                horizontalalignment='center', verticalalignment='center', fontsize=20)

im = ax_cm.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
ax_cm.set_title('Confusion Matrix', fontsize=18)

# Add value annotations
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax_cm.text(j, i, format(cm[i, j], 'd'),
                   ha="center", va="center",
                   color="white" if cm[i, j] > thresh else "black")

# Set labels
ax_cm.set_xlabel('Predicted label', fontsize=12)
ax_cm.set_ylabel('True label', fontsize=12)
ax_cm.set_xticks([0, 1])
ax_cm.set_yticks([0, 1])
ax_cm.set_xticklabels(['False', 'True'], fontsize=12)
ax_cm.set_yticklabels(['False', 'True'], fontsize=12)

# Add colorbar
plt.colorbar(im, ax=ax_cm)
plt.tight_layout()
plt.show()